# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119038e+02     1.715931e+00
 * time: 0.07577085494995117
     1     1.103532e+01     9.720456e-01
 * time: 1.391779899597168
     2    -1.139322e+01     9.877438e-01
 * time: 1.4919068813323975
     3    -3.375419e+01     8.642042e-01
 * time: 1.621243953704834
     4    -4.700773e+01     7.062267e-01
 * time: 1.775650978088379
     5    -5.667329e+01     2.673316e-01
 * time: 1.8930788040161133
     6    -5.954156e+01     2.949440e-01
 * time: 1.9897449016571045
     7    -6.079350e+01     1.303211e-01
 * time: 2.091615915298462
     8    -6.138066e+01     5.033727e-02
 * time: 2.2047038078308105
     9    -6.164814e+01     3.551597e-02
 * time: 2.2956769466400146
    10    -6.183533e+01     2.792023e-02
 * time: 2.386706829071045
    11    -6.197559e+01     2.444381e-02
 * time: 2.4791507720947266
    12    -6.207153e+01     1.968968e-02
 * time: 2.5702338218688965
    13    -6.211809e+01     1.669925e-02
 * time: 2.67524790763855

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671063
    AtomicLocal         -18.8557587
    AtomicNonlocal      14.8522571
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485355 
    Xc                  -19.3336815

    total               -62.261666458769
